In [1]:
import os
import requests
from io import StringIO
from functools import reduce
from random import sample
import pandas as pd

In [2]:
mult = lambda x, y: x * y

def IRA(W, V):
    """
    IRA: Indicador de recarga de acuifero, rango [1, 5]
    W: Porderadores de las variables (Resultado de procesar encuestas expertos), rango (0, 1)
    V: Valores de las variables para un lugar dado, rango [, ]
    """
    return sum(map(mult, W, V))

In [3]:
def var_list(prefix, n):
    """Genera nombres para columans e indices"""
    return [prefix + str(x + 1) for x in range(n)]    

In [4]:
def expert_survey(n):
    """Genera resultado encuesta de un experto para n variables"""
    return sample(range(1, n+1), n)

In [5]:
def data_survey(n=13, m=10):
    """Genera resultados de encuesta de m expertos para n varibles """
    # Genera un DataFrame vacio
    df = pd.DataFrame(pd.np.nan, index=var_list('v', n), columns=var_list('expert', m))
    # Rellena encuesta con resultados aleatorios
    for column in df:
        df[column] = expert_survey(n)
    return df

In [6]:
survey_exist = os.path.isfile('data.xlsx')

if survey_exist:
    # Leer excel guardado
    df = pd.read_excel('data.xlsx')
else:
    # Genera nueva encuesta de 10 expertos para 13 varibles
    df = data_survey(13, 10)
    # Guarda encuesta en un excel
    df.to_excel('data.xlsx')

# Muestra encuesta a utilizar
df

,expert1,expert2,expert3,expert4,expert5,expert6,expert7,expert8,expert9,expert10
v1,2,9,7,7,8,8,9,2,7,12
v2,3,6,10,6,3,1,4,11,3,13
v3,13,10,12,3,7,3,8,8,11,1
v4,9,3,6,8,12,4,13,10,5,6
v5,11,12,1,9,1,10,10,1,12,4
v6,5,1,9,4,5,6,7,4,1,5
v7,10,5,11,5,11,9,6,6,13,11
v8,8,13,5,10,4,7,2,7,9,2
v9,4,2,8,11,6,13,11,5,4,8
v10,6,7,13,2,13,11,1,13,2,10


In [7]:
def diff(x, y):
    return x - y

def Fc(df):
    """Calculo de la comparación por pares"""
    aux = pd.DataFrame()
    for index, row in df.iterrows():
        row_index = df.loc[index,:]
        k = df.apply(lambda row: diff(row, row_index), axis=1)
        m = k > 0
        l = k.mask(m, 1).mask(~m, 0)
        aux[index] = l.apply(sum)
    return aux.T

In [8]:
Fc(df)

,expert1,expert2,expert3,expert4,expert5,expert6,expert7,expert8,expert9,expert10
v1,11,4,6,6,5,5,4,11,6,1
v2,10,7,3,7,10,12,9,2,10,0
v3,0,3,1,10,6,10,5,5,2,12
v4,4,10,7,5,1,9,0,3,8,7
v5,2,1,12,4,12,3,3,12,1,9
v6,8,12,4,9,8,7,6,9,12,8
v7,3,8,2,8,2,4,7,7,0,2
v8,5,0,8,3,9,6,11,6,4,11
v9,9,11,5,2,7,0,2,8,9,5
v10,7,6,0,11,0,2,12,0,11,3


In [9]:
def Wt(df):
    """Calcula pesos relativos"""
    return df.apply(sum, axis=1) / reduce(mult, df.shape)

def W(s):
    """Calcula ponderaciones"""
    return s / sum(s)

In [10]:
Wt(Fc(df))

v1     0.453846
v2     0.538462
v3     0.415385
v4     0.415385
v5     0.453846
v6     0.638462
v7     0.330769
v8     0.484615
v9     0.446154
v10    0.400000
v11    0.461538
v12    0.415385
v13    0.546154
dtype: float64

In [11]:
# Calcula ponderaciones
w = W(Wt(Fc(df)))
w

v1     0.075641
v2     0.089744
v3     0.069231
v4     0.069231
v5     0.075641
v6     0.106410
v7     0.055128
v8     0.080769
v9     0.074359
v10    0.066667
v11    0.076923
v12    0.069231
v13    0.091026
dtype: float64

In [12]:
# Prueba que ponderaciones sumen 1
sum(w)

0.99999999999999989

In [13]:
# Guardar a Excel
w.to_excel('W.xlsx', header=['W'])
# Leer desde Excel
# pd.read_excel('W.xlsx')

In [14]:
# Prueba IRA
IRA(w, 1/w)

13.0

In [15]:
1/w

v1     13.220339
v2     11.142857
v3     14.444444
v4     14.444444
v5     13.220339
v6      9.397590
v7     18.139535
v8     12.380952
v9     13.448276
v10    15.000000
v11    13.000000
v12    14.444444
v13    10.985915
dtype: float64

In [16]:
# Leer datos desde resultados encuesta
r = requests.get('https://docs.google.com/spreadsheets/d/1ge_8d3gm0y35dXdrsRCQig4oo7hA-vCzFyNHwquzR8U/export?format=csv&gid=0')
data = r.content.decode('utf-8')
df = pd.read_csv(StringIO(data), index_col=0, parse_dates=['Timestamp'])
df

,Component [var1],Component [var2],Component [var3],Component [var4],Component [var5],Component [var6]
Timestamp,,,,,,
2017-09-13 12:57:18,1,2,3,4,5,6
2017-09-13 12:59:01,6,5,4,3,2,1
2017-09-13 13:04:28,1,3,2,5,4,6
2017-09-13 13:08:27,1,5,2,4,3,6


In [17]:
W(Wt(Fc(df.T)))

Component [var1]    0.250000
Component [var2]    0.150000
Component [var3]    0.216667
Component [var4]    0.133333
Component [var5]    0.166667
Component [var6]    0.083333
dtype: float64

In [18]:
# Guardar a Excel
w.to_excel('W_survey.xlsx', header=['W'])